In [1]:
!pip install tomotopy

In [2]:
import pandas as pd
import numpy as np
import tomotopy as tp
import re
import os
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [4]:
os.listdir('/gdrive/My Drive/court_files')

['פרטיות ',
 'חוזי שכירות  שלום ומחוזי',
 'ירושה ',
 'מקרקעין מעל 10 עמודים שלום ומחוזי']

In [5]:
m_corpus = tp.utils.Corpus()

topic_dirs = os.listdir('/gdrive/My Drive/court_files')

for topic_dir in topic_dirs:
    
    files = [os.path.abspath(f'/gdrive/My Drive/court_files/{topic_dir}/{file}') 
               for file in os.listdir(f'/gdrive/My Drive/court_files/{topic_dir}')]
    
    for raw_file in files:
        with open(raw_file) as input_f:
            file_txt = input_f.read()
            clean_txt = re.sub(r'[\W\d]+', ' ', file_txt)
            m_corpus.add_doc([s for s in clean_txt.split() if len(s) > 3], labels=[topic_dir])

In [23]:
# low alpha larger mixture of topics per doc 

plda = tp.PLDAModel(
    min_cf=10,
    min_df=3,
    rm_top=5,
    topics_per_label=1,
    latent_topics=0,
alpha = 0.2)

plda.add_corpus(m_corpus)

for i in range(0, 200, 20):
    plda.train(20)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, plda.ll_per_word))

Iteration: 0	Log-likelihood: -8.573022432364958
Iteration: 20	Log-likelihood: -8.573022186502099
Iteration: 40	Log-likelihood: -8.573022187440074
Iteration: 60	Log-likelihood: -8.573022188264353
Iteration: 80	Log-likelihood: -8.57302218741165
Iteration: 100	Log-likelihood: -8.573022187383227
Iteration: 120	Log-likelihood: -8.573022188264353
Iteration: 140	Log-likelihood: -8.573022187383227
Iteration: 160	Log-likelihood: -8.573022177321324
Iteration: 180	Log-likelihood: -8.573022187496921


In [24]:
test = 'למרות שבחוזה השכירות שלי היה כתוב שהאחריות היא על הבעלים  הצינור נשבר והוא ביקש ממני לשלם'

test2 = 'פורסמה כתבה ובה נטען כי אני מרמה את הלקוחות שלי, הדבר הוא שקר'

test3 = 'אמא שלי כתבה צוואה ולא הזכירה את השכן'

test4 = 'הדירה רשומה אצלי בטאבו ואני לא מצליח להעביר את הזכויות'

inferred_corpus, ll = plda.infer([
    plda.make_doc(words=text.split()) for text in [test, test2, test3, test4]
])

In [25]:
pd.options.display.float_format = '{:.2%}'.format

pd.DataFrame(inferred_corpus, columns=plda.topic_label_dict)

,פרטיות,חוזי שכירות שלום ומחוזי,ירושה,מקרקעין מעל 10 עמודים שלום ומחוזי
0,0.03%,99.89%,0.04%,0.05%
1,0.04%,0.06%,0.05%,99.85%
2,0.08%,0.12%,99.68%,0.12%
3,0.03%,99.89%,0.04%,0.05%
